In [ ]:
import imutils
import face_recognition
import cv2
from scipy.spatial import distance as dist
import playsound
from threading import Thread
import numpy as np
import os
MIN_AER =0.30  #min aer to tell person closed eyes or not
EYE_AR_CONSEC_FRAMES =10 #after 10 frames alaram on
COUNTER = 0 #to count frames
ALARM_ON = False
def playAlarm(soundfile): #play alaram
    playsound.playsound(soundfile)

def eye_aspect_ratio(eye):  #to determine if eyes are closed
    V1 = dist.euclidean(eye[1],eye[5])
    V2 = dist.euclidean(eye[2], eye[4])
    H = dist.euclidean(eye[0], eye[3])
    ear = (V1+V2)/(2.0*H)
    return ear

def main():
    global COUNTER,ALARM_ON
    video_capture = cv2.VideoCapture(0)
    video_capture.set(3,320)
    video_capture.set(4,240)
    while True:
        ret ,frame = video_capture.read()
        face_landmarks_list = face_recognition.face_landmarks(frame)
        for face_landmark in face_landmarks_list:
            leftEye = face_landmark["left_eye"] #left eye landmarks
            rightEye = face_landmark["right_eye"] #right eye landmarks
            lpts = np.array(leftEye) #left eye points
            rpts = np.array(rightEye)

            cv2.polylines(frame,[lpts],True,(255,255,0),1)  #drawing points on left eye
            cv2.polylines(frame, [rpts], True, (255, 255, 0) #drawing points on  right eye

            leftEAR = eye_aspect_ratio(leftEye) #calling and cal ear respectively
            rightEAR = eye_aspect_ratio(rightEye)

            ear = (leftEAR+rightEAR)/2 #avg

            if ear<MIN_AER: #if less taen  min area
                COUNTER+=1 #count how many frames
                if COUNTER>=EYE_AR_CONSEC_FRAMES:  #exced more than 10 frames
                    if not ALARM_ON:
                        ALARM_ON = True  #on alaram
                        t = Thread(target=playAlarm,args=('alarm.wav',))
                        t.daemon = True
                        t.start()
                    cv2.putText(frame,"ALERT! You are feeling asleep!",(10,10), #keeping of text
                                cv2.FONT_HERSHEY_SIMPLEX,0.4,(0,0,255),1)
            else:
                COUNTER=0 
                ALARM_ON = False
            cv2.putText(frame, "EAR: {:.2f}".format(ear), (250, 10),  #print ear value
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
            cv2.imshow("Sleep detection system",frame)
        if cv2.waitKey(1)== ord('q'):
            break
    video_capture.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()